In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from preprocess import DB
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [3]:
db = DB(db_route = '../data/database.sqlite')

In [8]:
df = db.combineCardFoulDF()

  0%|          | 0/14217 [00:00<?, ?it/s]

In [10]:
df.to_csv('../data/card_foul.csv')

In [14]:
df2 = pd.merge(df, db.match_team, how = 'left', left_on = 'match_id', right_on = 'match_api_id').drop(['foulcommit', 'card', 'corner'], axis = 1)

In [21]:
df2[df2['match_id'] == 489042]

,elapsed_foul,event_incident_typefk_foul,foul_id,injury_time,player1_id_foul,player2_id_foul,subtype,team,type,match_id,...,player1_id_card,card_subtype,team_card,card_type,match_api_id,date,home_team_api_id,home_team_name,away_team_api_id,away_team_name
0,0,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
1,1,37,378824,0,25518,32569,0,10261,foulcommit,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
2,2,37,378826,0,30929,24157,0,10261,foulcommit,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
3,3,37,378830,0,29581,24148,0,10261,foulcommit,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
4,4,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,115,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
117,116,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
118,117,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
119,118,0,0,0,0,0,NaN,0,NaN,489042,...,NaN,NaN,NaN,NaN,489042,2008-08-17 00:00:00,10260,Manchester United,10261,Newcastle United
